# Flower Recognition model using CNN

In [1]:
import os
import cv2
import numpy as np
from PIL import Image

#Encoding and Split data into Train/Test Sets
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

#Tensorflow Keras CNN Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

#Plot Images
import matplotlib.pyplot as plt


folder_dir = 'C:/Users/Administrator/Documents/flowers'

Labelling images is an important task beroe model fitting, as appropriate annotions will help our model classify different types of flower.
In order to create our dataset, we have to real all the images from each flower type we are going to classify.
Each folder within the directories should contain flowers according to the folder name as we are going to use the names for the annotation.

# Adding images and labels to our data

In [2]:
data = []
label = []

SIZE = 128 #Crop the image to 128x128

for folder in os.listdir(folder_dir):
    for file in os.listdir(os.path.join(folder_dir, folder)):
        if file.endswith("jpg"):
            label.append(folder)
            img = cv2.imread(os.path.join(folder_dir, folder, file))
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            im = cv2.resize(img_rgb, (SIZE,SIZE))
            data.append(im)
        else:
            continue

In [3]:
#pixel values of first 20 images
data[:20]

[array([[[137, 137, 135],
         [145, 145, 145],
         [148, 148, 149],
         ...,
         [153, 153, 151],
         [154, 154, 152],
         [154, 154, 154]],
 
        [[135, 135, 133],
         [144, 144, 143],
         [147, 147, 147],
         ...,
         [154, 154, 152],
         [153, 153, 151],
         [151, 151, 151]],
 
        [[134, 133, 129],
         [142, 141, 139],
         [146, 145, 143],
         ...,
         [156, 156, 155],
         [152, 152, 150],
         [148, 148, 148]],
 
        ...,
 
        [[ 43,  46,  25],
         [ 43,  46,  25],
         [ 43,  46,  25],
         ...,
         [129, 125, 124],
         [128, 124, 123],
         [130, 126, 125]],
 
        [[ 44,  47,  26],
         [ 44,  47,  26],
         [ 45,  48,  27],
         ...,
         [132, 128, 127],
         [130, 126, 125],
         [130, 126, 125]],
 
        [[ 44,  47,  26],
         [ 44,  47,  26],
         [ 45,  48,  27],
         ...,
         [133, 129, 128],
  

Converting our image to numpy array to get the pixel values

In [4]:
data_arr = np.array(data)
label_arr = np.array(label)

Label encoding our labels and normalizing our pixel values to the range of 0-1

In [5]:
encoder = LabelEncoder()
y = encoder.fit_transform(label_arr)
y = to_categorical(y,5)
X = data_arr/255

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=10)

# Using a Convolutional Neural Network to help our model classify flower types based on pixel values

In [7]:
model = Sequential()
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu', input_shape = (SIZE,SIZE,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(5, activation = "softmax"))

Before compiling the model we need to create more training images to prevent overfitting:

In [8]:
datagen = ImageDataGenerator(
        rotation_range=20,
        zoom_range = 0.20,
        width_shift_range=0.3,
        height_shift_range=0.3,
        horizontal_flip=True,
        vertical_flip=True)

datagen.fit(X_train)

# Running multiple epochs to increase the accuracy score

In [9]:
model.compile(optimizer=Adam(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
batch_size=32
epochs=5
history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs,
                              validation_data = (X_test,y_test),
                              verbose = 1)

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
108/108 [==============================] - 132s 1s/step - loss: 1.4819 - accuracy: 0.3267 - val_loss: 1.2628 - val_accuracy: 0.5023
Epoch 2/5
108/108 [==============================] - 128s 1s/step - loss: 1.3279 - accuracy: 0.4382 - val_loss: 1.1931 - val_accuracy: 0.5509
Epoch 3/5
108/108 [==============================] - 128s 1s/step - loss: 1.2352 - accuracy: 0.4894 - val_loss: 1.1298 - val_accuracy: 0.5370
Epoch 4/5
108/108 [==============================] - 129s 1s/step - loss: 1.1767 - accuracy: 0.5172 - val_loss: 1.1158 - val_accuracy: 0.5509
Epoch 5/5
108/108 [==============================] - 130s 1s/step - loss: 1.1558 - accuracy: 0.5450 - val_loss: 1.1415 - val_accuracy: 0.5521


Accuracy score can be increased further by running more epochs.
Running more epochs can will increase the time significantly for the model to complete execution

In [ ]:
categories = np.sort(os.listdir(folder_dir))
fig, ax = plt.subplots(6,6, figsize=(25, 40))

for i in range(6):
    for j in range(6):
        k = int(np.random.random_sample() * len(X_test))
        if(categories[np.argmax(y_test[k])] == categories[np.argmax(model.predict(X_test)[k])]):
            ax[i,j].set_title("TRUE: " + categories[np.argmax(y_test[k])], color='green')
            ax[i,j].set_xlabel("PREDICTED: " + categories[np.argmax(model.predict(X_test)[k])], color='green')
            ax[i,j].imshow(np.array(X_test)[k].reshape(SIZE, SIZE, 3), cmap='gray')
        else:
            ax[i,j].set_title("TRUE: " + categories[np.argmax(y_test[k])], color='red')
            ax[i,j].set_xlabel("PREDICTED: " + categories[np.argmax(model.predict(X_test)[k])], color='red')
            ax[i,j].imshow(np.array(X_test)[k].reshape(SIZE, SIZE, 3), cmap='gray')